Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `'bank_marketing.csv'`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `'.'` to `'_'` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `'.'` to `'_'` and `'unknown'` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `'yes'`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `'yes'`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `'success'`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `'yes'`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `'YYYY-MM-DD'` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [55]:
# Load libraries
import pandas as pd
import numpy as np

In [56]:
# Load data

marketing = pd.read_csv('bank_marketing.csv')

print(marketing.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')


In [57]:
# Create three new DataFrames
client = marketing[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]

campaign = marketing[['client_id', 'number_contacts', 'month', 'day', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]

economics = marketing[['client_id', 'cons_price_idx', 'euribor_three_months']]

In [58]:
# Inspect data

display(client.head())
display(campaign.head())
display(economics.head())

for col in ['credit_default', 'mortgage', 'previous_outcome', 'campaign_outcome']:
    print(col)
    print('--------------')
    print(marketing[col].value_counts())

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


,client_id,number_contacts,month,day,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,0,nonexistent,no
1,1,1,may,19,149,0,nonexistent,no
2,2,1,may,23,226,0,nonexistent,no
3,3,1,may,27,151,0,nonexistent,no
4,4,1,may,3,307,0,nonexistent,no


,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


credit_default
--------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
--------------
mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64
previous_outcome
--------------
previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64
campaign_outcome
--------------
campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64


In [59]:
# Replace values in 'education' containing '.' with '_'
client['education'] = client['education'].str.replace('.', '_')

# Change 'unknown' values to null values (represented as np.NaN) in the 'education' column.
client['education'] = client['education'].replace('unknown', np.NaN)

C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\1575621063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['education'] = client['education'].str.replace('.', '_')
C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\1575621063.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['education'] = client['education'].replace('unknown', np.NaN)


In [60]:
# The 'job' column should have periods removed, which can be done by replacing '.' with an empty string ''.
client['job'] = client['job'].str.replace('.','')

C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\431019312.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['job'] = client['job'].str.replace('.','')


In [61]:
# Convert 'campaign_outcome' to binary values
for col in ['credit_default', 'mortgage']:
  client[col] = client[col].map({'yes': 1,
                                 'no': 0,
                                 'unknown': 0})
  client[col] = client[col].astype(bool)

# Convert 'campaign_outcome' to binary values
# campaign['campaign_outcome'] = campaign['campaign_outcome'].astype(bool)
print(campaign.dtypes)

client_id                      int64
number_contacts                int64
month                         object
day                            int64
contact_duration               int64
previous_campaign_contacts     int64
previous_outcome              object
campaign_outcome              object
dtype: object


C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\1784130619.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client[col] = client[col].map({'yes': 1,
C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\1784130619.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client[col] = client[col].astype(bool)
C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\1784130619.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [62]:
# Replace values in 'previous_outcome': 'success' to 1 or True, and from 'failure'/'nonexistent' to 0 or False respectively
campaign['previous_outcome'] = campaign['previous_outcome'].map({'success': 1, 
                                                                 'failure': 0,
                                                                 'nonexistent': 0})

C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\2414188141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['previous_outcome'] = campaign['previous_outcome'].map({'success': 1,


In [63]:
# Add a new column called 'year' where all values are set as '2022' (note this should be a string)
campaign['year'] = '2022'

# Convert the 'day' column to a string
campaign['day'] = campaign['day'].astype(str)

# Add last_contact_date column
campaign['last_contact_date'] = campaign['year'] + '-' + campaign['month'] + '-' + campaign['day']

# Convert last_contact_date column to datetime format

campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], format = '%Y-%b-%d')

C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\2491219043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['year'] = '2022'
C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\2491219043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['day'] = campaign['day'].astype(str)
C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\2491219043.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [64]:
# Clean and convert outcome columns to bool
for col in ['campaign_outcome','previous_outcome']:
  campaign[col] = campaign[col].astype(bool)

# Drop unneccessary columns
campaign.drop(columns=['month', 'day', 'year'], inplace=True)

# Save tables to individual csv files
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\2585108289.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign[col] = campaign[col].astype(bool)
C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\2585108289.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign[col] = campaign[col].astype(bool)
C:\Users\tkirov\AppData\Local\Temp\ipykernel_27496\2585108289.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 